In [1]:
import org.apache.spark.rdd.RDD
import scala.collection.Map

import org.apache.spark.{SparkContext, SparkConf}
import org.apache.spark.sql.{DataFrame, Row, SQLContext}
import org.apache.spark.sql.types._
    
import scala.collection.immutable.WrappedString
import scala.collection.Map

**Préparer Spark**

In [2]:
val sparkConf = new SparkConf().setMaster("local").setAppName("TF-IDF")
val sc = new SparkContext(sparkConf)

sparkConf = org.apache.spark.SparkConf@f64aeed
sc = org.apache.spark.SparkContext@6e18a803


org.apache.spark.SparkContext@6e18a803

**Declarer les endroits qui puissent avoir les informations nécessaires**

In [3]:
val in = "./data_tfidf/bbctext/tech"
val in_stop_words = "./data_tfidf/stop_words"

in = ./data_tfidf/bbctext/tech
in_stop_words = ./data_tfidf/stop_words


./data_tfidf/stop_words

**Lire tous les documents dans le folder et les ramasser dans une variable**

In [4]:
val bbctext = sc.wholeTextFiles(in)
// bbctext.take(1).foreach(println)

bbctext = ./data_tfidf/bbctext/tech MapPartitionsRDD[1] at wholeTextFiles at <console>:38


./data_tfidf/bbctext/tech MapPartitionsRDD[1] at wholeTextFiles at <console>:38

**Creer une classe pour pouvoir traiter les données plus tard aec un dataframe et sql spark**

In [5]:
case class File_Word(file: String, word: String, count: Int)

defined class File_Word


**PREMIERE VARIABLE: N = quantité de documents**

In [6]:
var N = bbctext.count()

N = 401


401

**Mapper tous les mots existants en suivant la structure suivante pour chaque mot: (mot, nom_du_fichier_contenant_ce_mot, 1)**

In [7]:
val words = bbctext.flatMap{ case (file_path, text) => {
    // Premièrement, séparer tous les mots et les faire devenir lowercase
    var all_words = text.toLowerCase.split("""[^\p{IsAlphabetic}]+""") 
    var file_name = file_path.split("/bbctext/")(1)
    // Deuxièmement, assign value for 
    var file_word = all_words.map(word => ((word, file_name), 1))
    file_word
}}

words = MapPartitionsRDD[2] at flatMap at <console>:36


MapPartitionsRDD[2] at flatMap at <console>:36

In [8]:
words.take(1).foreach(println)

((ink,tech/001.txt),1)


**Grace a la structure créée, on peut sommer les mots pour savoir combien de chaque mot, il y a dans chaque document**

In [9]:
val words_on_files = words.reduceByKey(_ + _)

words_on_files = ShuffledRDD[3] at reduceByKey at <console>:36


ShuffledRDD[3] at reduceByKey at <console>:36

In [10]:
words_on_files.take(5).foreach(println)

((pie,tech/009.txt),1)
((nasa,tech/173.txt),1)
((supplies,tech/142.txt),1)
((simmelink,tech/350.txt),1)
((concluded,tech/361.txt),1)


**On cree une classe pour contenir les mots que l'on veut enlever et on lit le fichier**

In [11]:
case class Stop_Word(sword: String)
var stop_words = sc.textFile(in_stop_words)

defined class Stop_Word
stop_words = ./data_tfidf/stop_words MapPartitionsRDD[5] at textFile at <console>:18


./data_tfidf/stop_words MapPartitionsRDD[5] at textFile at <console>:18

In [12]:
stop_words.take(2).foreach(println)

a
about


**Récuperer la séquence des mots à étudier, dans la forme (file, word, count) pour faciliter le traitement**

In [13]:
var files_words = words_on_files.map{ case((word, file), count) => 

    File_Word(file, word, count)
}

files_words = MapPartitionsRDD[6] at map at <console>:38


MapPartitionsRDD[6] at map at <console>:38

In [14]:
files_words.take(1).foreach(println)

File_Word(tech/009.txt,pie,1)


**Manipuler les mots ne pas désirés pour les avoir dans la même forme que les mots à étudier**

In [15]:
var stop_split = stop_words.flatMap(line => line.split("\\W+"))
stop_split.take(5).foreach(println)

a
about
above
across
after


stop_split = MapPartitionsRDD[7] at flatMap at <console>:36


MapPartitionsRDD[7] at flatMap at <console>:36

**Convertir les deux, stop et mots à étudier en spark dataframes pour les traiter avec les outils spark**

In [16]:
var df_files_words = spark.createDataFrame(files_words)

df_files_words = [file: string, word: string ... 1 more field]


[file: string, word: string ... 1 more field]

In [17]:
var ds_stop_words = stop_split.toDS()
var df_stop_words = ds_stop_words.map{ word => Stop_Word(word)}

ds_stop_words = [value: string]
df_stop_words = [sword: string]


[sword: string]

**Faire le join entre les deux tas de data et prendre juste les données nécessaires (left-join)**

In [18]:
var join_fileswrd_stopword = df_files_words.join(df_stop_words, 
                                            df_files_words("word") === df_stop_words("sword"), 
                                            "left_outer")
join_fileswrd_stopword.show(100)

+------------+---------------+-----+-----+
|        file|           word|count|sword|
+------------+---------------+-----+-----+
|tech/019.txt|      amplifier|    1| null|
|tech/329.txt|      amplifier|    1| null|
|tech/032.txt|       antennae|    2| null|
|tech/335.txt|       antennae|    2| null|
|tech/205.txt|      arguments|    1| null|
|tech/070.txt|      arguments|    1| null|
|tech/264.txt|      arguments|    1| null|
|tech/378.txt|            art|    1| null|
|tech/005.txt|            art|    7| null|
|tech/107.txt|            art|    1| null|
|tech/057.txt|            art|    1| null|
|tech/388.txt|            art|    1| null|
|tech/056.txt|            art|    1| null|
|tech/236.txt|            art|    1| null|
|tech/113.txt|            art|    1| null|
|tech/103.txt|            art|    1| null|
|tech/105.txt|            art|    1| null|
|tech/396.txt|            art|    2| null|
|tech/085.txt|            art|    4| null|
|tech/281.txt|        barrier|    1| null|
|tech/386.t

join_fileswrd_stopword = [file: string, word: string ... 2 more fields]


[file: string, word: string ... 2 more fields]

In [19]:
join_fileswrd_stopword.createOrReplaceTempView("join_fileswrd_stopword")

In [20]:
var clean_files_words = 
    spark.sql("SELECT * FROM join_fileswrd_stopword WHERE join_fileswrd_stopword.sword IS NULL")
clean_files_words.show(100)

+------------+---------------+-----+-----+
|        file|           word|count|sword|
+------------+---------------+-----+-----+
|tech/019.txt|      amplifier|    1| null|
|tech/329.txt|      amplifier|    1| null|
|tech/032.txt|       antennae|    2| null|
|tech/335.txt|       antennae|    2| null|
|tech/205.txt|      arguments|    1| null|
|tech/070.txt|      arguments|    1| null|
|tech/264.txt|      arguments|    1| null|
|tech/378.txt|            art|    1| null|
|tech/005.txt|            art|    7| null|
|tech/107.txt|            art|    1| null|
|tech/057.txt|            art|    1| null|
|tech/388.txt|            art|    1| null|
|tech/056.txt|            art|    1| null|
|tech/236.txt|            art|    1| null|
|tech/113.txt|            art|    1| null|
|tech/103.txt|            art|    1| null|
|tech/105.txt|            art|    1| null|
|tech/396.txt|            art|    2| null|
|tech/085.txt|            art|    4| null|
|tech/281.txt|        barrier|    1| null|
|tech/386.t

clean_files_words = [file: string, word: string ... 2 more fields]


[file: string, word: string ... 2 more fields]

In [21]:
clean_files_words = clean_files_words.drop("sword")
//clean_files_words.cache()
clean_files_words.show(2)

+------------+---------+-----+
|        file|     word|count|
+------------+---------+-----+
|tech/019.txt|amplifier|    1|
|tech/329.txt|amplifier|    1|
+------------+---------+-----+
only showing top 2 rows



clean_files_words = [file: string, word: string ... 1 more field]


[file: string, word: string ... 1 more field]

**Obtenir les functions spark pour les méthodes d'aggrégation**

In [22]:
import org.apache.spark.sql.functions._

**obtenir le nombre de mots relevants par document**

In [23]:
var relevant_words_per_file = clean_files_words.groupBy("file").agg(sum(clean_files_words("count")).as("rlvnt_wrds"))
//relevant_words_per_file.cache()
relevant_words_per_file = relevant_words_per_file.withColumnRenamed("file", "relevant_file")
relevant_words_per_file.show(5)

+-------------+----------+
|relevant_file|rlvnt_wrds|
+-------------+----------+
| tech/381.txt|       234|
| tech/003.txt|       122|
| tech/114.txt|       209|
| tech/186.txt|       182|
| tech/182.txt|       119|
+-------------+----------+
only showing top 5 rows



relevant_words_per_file = [relevant_file: string, rlvnt_wrds: bigint]
relevant_words_per_file = [relevant_file: string, rlvnt_wrds: bigint]


[relevant_file: string, rlvnt_wrds: bigint]

**Obtenir la table avec la quantité de repetitions par document des mots et parallèlement la quantité totale de mots relevants par document**

In [24]:
var terms_occurrences = clean_files_words.join(relevant_words_per_file, 
                                              relevant_words_per_file("relevant_file") === clean_files_words("file"))
terms_occurrences.show(100)

+------------+-------------+-----+-------------+----------+
|        file|         word|count|relevant_file|rlvnt_wrds|
+------------+-------------+-----+-------------+----------+
|tech/003.txt|       online|    1| tech/003.txt|       122|
|tech/003.txt|      spyware|    6| tech/003.txt|       122|
|tech/003.txt|         anti|    6| tech/003.txt|       122|
|tech/003.txt|         slow|    1| tech/003.txt|       122|
|tech/003.txt|      earlier|    1| tech/003.txt|       122|
|tech/003.txt|    microsoft|    9| tech/003.txt|       122|
|tech/003.txt|          buy|    1| tech/003.txt|       122|
|tech/003.txt|          set|    1| tech/003.txt|       122|
|tech/003.txt|       causes|    1| tech/003.txt|       122|
|tech/003.txt|          pop|    1| tech/003.txt|       122|
|tech/003.txt|     programs|    1| tech/003.txt|       122|
|tech/003.txt|         form|    1| tech/003.txt|       122|
|tech/003.txt|       sybari|    1| tech/003.txt|       122|
|tech/003.txt|          own|    1| tech/

terms_occurrences = [file: string, word: string ... 3 more fields]


[file: string, word: string ... 3 more fields]

**NORMALISER**

In [25]:
var occurrences_normalized = terms_occurrences.withColumn("tf", expr("count/rlvnt_wrds"))
occurrences_normalized = occurrences_normalized.drop("relevant_file")
occurrences_normalized.show(5)

+------------+-------+-----+----------+-------------------+
|        file|   word|count|rlvnt_wrds|                 tf|
+------------+-------+-----+----------+-------------------+
|tech/003.txt| online|    1|       122|0.00819672131147541|
|tech/003.txt|spyware|    6|       122|0.04918032786885246|
|tech/003.txt|   anti|    6|       122|0.04918032786885246|
|tech/003.txt|   slow|    1|       122|0.00819672131147541|
|tech/003.txt|earlier|    1|       122|0.00819672131147541|
+------------+-------+-----+----------+-------------------+
only showing top 5 rows



occurrences_normalized = [file: string, word: string ... 3 more fields]
occurrences_normalized = [file: string, word: string ... 3 more fields]


[file: string, word: string ... 3 more fields]

**Par mot, obtenir les fois que celui ci se repete par document et la quantité de documents où c'est mentionné**

In [26]:
var word_in_doc = clean_files_words.groupBy("word")
    .agg(
        count(clean_files_words("file")).as("count_files"), 
        sum(clean_files_words("count")).as("sum_words")
    )
word_in_doc = word_in_doc.withColumnRenamed("word", "group_word")
word_in_doc.show(5)

+----------+-----------+---------+
|group_word|count_files|sum_words|
+----------+-----------+---------+
| amplifier|          2|        2|
|  antennae|          2|        4|
| arguments|          3|        3|
|       art|         12|       22|
|   barrier|         10|       12|
+----------+-----------+---------+
only showing top 5 rows



word_in_doc = [group_word: string, count_files: bigint ... 1 more field]
word_in_doc = [group_word: string, count_files: bigint ... 1 more field]


[group_word: string, count_files: bigint ... 1 more field]

**Joindre les mots normalisés avec la table du compteur de mots /apparition /document**

In [27]:
var join_wordsInDoc_ocurrences = occurrences_normalized.join(word_in_doc, 
                                                             word_in_doc("group_word") === occurrences_normalized("word"),
                                                            "inner")
join_wordsInDoc_ocurrences.show(5)

+------------+---------+-----+----------+--------------------+----------+-----------+---------+
|        file|     word|count|rlvnt_wrds|                  tf|group_word|count_files|sum_words|
+------------+---------+-----+----------+--------------------+----------+-----------+---------+
|tech/019.txt|amplifier|    1|       220|0.004545454545454545| amplifier|          2|        2|
|tech/329.txt|amplifier|    1|       220|0.004545454545454545| amplifier|          2|        2|
|tech/335.txt| antennae|    2|       421|0.004750593824228029|  antennae|          2|        4|
|tech/032.txt| antennae|    2|       426|0.004694835680751174|  antennae|          2|        4|
|tech/205.txt|arguments|    1|       197|0.005076142131979695| arguments|          3|        3|
+------------+---------+-----+----------+--------------------+----------+-----------+---------+
only showing top 5 rows



join_wordsInDoc_ocurrences = [file: string, word: string ... 6 more fields]


[file: string, word: string ... 6 more fields]

**---> TFIDF :)**

In [28]:
var tfidf_table = join_wordsInDoc_ocurrences
    .drop("group_word")
    .withColumn("tfidf", expr("tf * log("+N+"/count_files)"))
tfidf_table.show(100)

+------------+---------------+-----+----------+--------------------+-----------+---------+--------------------+
|        file|           word|count|rlvnt_wrds|                  tf|count_files|sum_words|               tfidf|
+------------+---------------+-----+----------+--------------------+-----------+---------+--------------------+
|tech/019.txt|      amplifier|    1|       220|0.004545454545454545|          2|        2|0.024094610212484654|
|tech/329.txt|      amplifier|    1|       220|0.004545454545454545|          2|        2|0.024094610212484654|
|tech/335.txt|       antennae|    2|       421|0.004750593824228029|          2|        4|0.025182015423974462|
|tech/032.txt|       antennae|    2|       426|0.004694835680751174|          2|        4|0.024886451862660208|
|tech/205.txt|      arguments|    1|       197|0.005076142131979695|          3|        3|0.024849488013393192|
|tech/264.txt|      arguments|    1|       349|0.002865329512893...|          3|        3|0.014026788362

tfidf_table = [file: string, word: string ... 6 more fields]


[file: string, word: string ... 6 more fields]

**Ordonner tfidf table par file -> tfidf -> word**

In [29]:
var file_word_tfidf = tfidf_table.orderBy(col("file").asc, col("tfidf").desc, col("word").asc)
file_word_tfidf.show(100)

+------------+-------------+-----+----------+--------------------+-----------+---------+--------------------+
|        file|         word|count|rlvnt_wrds|                  tf|count_files|sum_words|               tfidf|
+------------+-------------+-----+----------+--------------------+-----------+---------+--------------------+
|tech/001.txt|          ink|   23|       310| 0.07419354838709677|          5|       35| 0.32530335755505413|
|tech/001.txt|    elections|   12|       310| 0.03870967741935484|          2|       13| 0.20519280955148222|
|tech/001.txt|       kyrgyz|    4|       310|0.012903225806451613|          1|        4| 0.07734143777169766|
|tech/001.txt|        voter|    4|       310|0.012903225806451613|          1|        4| 0.07734143777169766|
|tech/001.txt|       ballot|    3|       310| 0.00967741935483871|          1|        3|0.058006078328773254|
|tech/001.txt|  ultraviolet|    3|       310| 0.00967741935483871|          1|        3|0.058006078328773254|
|tech/001.

file_word_tfidf = [file: string, word: string ... 6 more fields]


[file: string, word: string ... 6 more fields]

**Quels sont les mots les plus représentatifs par document (plus haut tfidf)?**

In [30]:
var representative_words_per_file = tfidf_table.groupBy("file").agg(max("tfidf"), max("word").as("word"))
representative_words_per_file.show(100)

+------------+-------------------+------------+
|        file|         max(tfidf)|        word|
+------------+-------------------+------------+
|tech/003.txt| 0.1526355575119871|     windows|
|tech/114.txt| 0.3043529711050693|     weekend|
|tech/186.txt|0.14443565829469712|       world|
|tech/381.txt|0.12294243263275895|       world|
|tech/182.txt|0.17817863014274365|       watch|
|tech/026.txt| 0.1490874812976101|     worried|
|tech/241.txt|0.15411495612703674|    websites|
|tech/306.txt| 0.1512908090893417|     website|
|tech/018.txt|0.14833002411786617|workstations|
|tech/080.txt|0.14496715833633267|       vital|
|tech/273.txt|0.12938976688474332|  wrongdoing|
|tech/341.txt|0.13252035616866562|          yh|
|tech/009.txt|0.06041777210458689|      zenith|
|tech/013.txt|0.14434645316452574|       worth|
|tech/008.txt|0.20558978844659317|        zafi|
|tech/160.txt|0.09009845040438882|     yarnton|
|tech/051.txt|0.22625426662942907|        york|
|tech/210.txt| 0.1906206814337778|      

representative_words_per_file = [file: string, max(tfidf): double ... 1 more field]


[file: string, max(tfidf): double ... 1 more field]

**Par mot, quel est le fichier où il se repete le plus et sa valeur tfidf?**

In [31]:
var most_repeated_words = tfidf_table.groupBy("word").agg(max("tfidf"), max("file").as("in_file"))
most_repeated_words.show(100)

+---------------+--------------------+------------+
|           word|          max(tfidf)|     in_file|
+---------------+--------------------+------------+
|      amplifier|0.024094610212484654|tech/329.txt|
|       antennae|0.025182015423974462|tech/335.txt|
|      arguments|0.029138982968086067|tech/264.txt|
|            art|  0.0806679259199671|tech/396.txt|
|        barrier| 0.03155022507959422|tech/386.txt|
|    battlefront|  0.1257820329736487|tech/125.txt|
|         brands| 0.10026133196907136|tech/364.txt|
|        bricked|0.016287938661159157|tech/005.txt|
|       carnegie| 0.03643745548514632|tech/199.txt|
|       cautious| 0.02236552771383048|tech/316.txt|
|            ces| 0.04627453927850084|tech/296.txt|
|     commanders| 0.01341978290315601|tech/311.txt|
|confidentiality|0.029490055052038913|tech/368.txt|
|      connected|0.030679740048255123|tech/395.txt|
|     creativity|0.028808773080144696|tech/309.txt|
|           cubs|0.033864188854839376|tech/107.txt|
|          c

most_repeated_words = [word: string, max(tfidf): double ... 1 more field]


[word: string, max(tfidf): double ... 1 more field]

**Télécharger les fichier pertinents pour faire de la visualisation, metre disponibles ses methods à la feulle de travail et déclarer une fonction pour montrer les plots**

In [32]:
// https://www.vegas-viz.org/
// https://github.com/vegas-viz/Vegas
%AddDeps org.vegas-viz vegas_2.11 0.3.11 --transitive
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.11

Marking org.vegas-viz:vegas_2.11:0.3.11 for download
Obtained 42 files
Marking org.vegas-viz:vegas-spark_2.11:0.3.11 for download
Obtained 2 files


In [33]:
import vegas._
import vegas.sparkExt._

implicit val render_vegas = vegas.render.ShowHTML(kernel.display.content("text/html", _))

render_vegas = <function1>


<function1>

**Combien de mots relevants per document**

In [34]:
 val RelevantWordsNotNormalized = Vegas("RelevantWordsNotNormalized")
      .withDataFrame(relevant_words_per_file)
      .encodeX("relevant_file", Nom)
      .encodeY("rlvnt_wrds", Quant)
      .mark(Bar)
render_vegas(RelevantWordsNotNormalized)

<iframe id="frame-vegas-21151018-56a7-4de9-905c-89c2048929f2" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-21151018-56a7-4de9-905c-89c2048929f2'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "relevant_file",
 "type" : "nominal"
 },
 "y" : {
 "field" : "rlvnt_wrds",
 "type" : "quantitative"
 }
 },
 "description" : "RelevantWordsNotNormalized",
 "data" : {
 "values" : [
 {
 "relevant_file" : "tech/381.txt",
 "rlvnt_wrds" : 234
 },
 {
 "relevant_file" : "tech/003.txt",
 "rlvnt_wrds" : 122
 },
 {
 "relevant_file" : "tech/114.txt",
 "rlvnt_wrds" : 209
 },
 {
 "relevant_file" : "tech/186.txt",
 "rlvnt_wrds" : 182
 },
 {
 "relevant_file" : "tech/182.txt",
 "rlvnt_wrds" : 119
 },
 {
 "relevant_file" : "tech/026.txt",
 "rlvnt_wrds" : 230
 },
 {
 "relevant_file" : "tech/241.txt",
 "rlvnt_wrds" : 127
 },
 {
 "relevant_file" : "tech/306.txt",
 "rlvnt_wrds" : 309
 },
 {
 "relevant_file" : "tech/341.txt",
 "rlvnt_wrds" : 120
 },
 {
 "relevant_file" : "tech/018.txt",
 "rlvnt_wrds" : 166
 },
 {
 "relevant_file" : "tech/273.txt",
 "rlvnt_wrds" : 219
 },
 {
 "relevant_file" : "tech/080.txt",
 "rlvnt_wrds" : 134
 },
 {
 "relevant_file" : "tech/009.txt",
 "rlvnt_wrds" : 540
 },
 {
 "relevant_file" : "tech/013.txt",
 "rlvnt_wrds" : 243
 },
 {
 "relevant_file" : "tech/008.txt",
 "rlvnt_wrds" : 149
 },
 {
 "relevant_file" : "tech/160.txt",
 "rlvnt_wrds" : 163
 },
 {
 "relevant_file" : "tech/210.txt",
 "rlvnt_wrds" : 283
 },
 {
 "relevant_file" : "tech/051.txt",
 "rlvnt_wrds" : 164
 },
 {
 "relevant_file" : "tech/313.txt",
 "rlvnt_wrds" : 366
 },
 {
 "relevant_file" : "tech/096.txt",
 "rlvnt_wrds" : 155
 },
 {
 "relevant_file" : "tech/344.txt",
 "rlvnt_wrds" : 133
 },
 {
 "relevant_file" : "tech/102.txt",
 "rlvnt_wrds" : 294
 },
 {
 "relevant_file" : "tech/014.txt",
 "rlvnt_wrds" : 139
 },
 {
 "relevant_file" : "tech/100.txt",
 "rlvnt_wrds" : 146
 },
 {
 "relevant_file" : "tech/122.txt",
 "rlvnt_wrds" : 301
 },
 {
 "relevant_file" : "tech/005.txt",
 "rlvnt_wrds" : 368
 },
 {
 "relevant_file" : "tech/073.txt",
 "rlvnt_wrds" : 288
 },
 {
 "relevant_file" : "tech/301.txt",
 "rlvnt_wrds" : 505
 },
 {
 "relevant_file" : "tech/335.txt",
 "rlvnt_wrds" : 421
 },
 {
 "relevant_file" : "tech/377.txt",
 "rlvnt_wrds" : 155
 },
 {
 "relevant_file" : "tech/035.txt",
 "rlvnt_wrds" : 213
 },
 {
 "relevant_file" : "tech/058.txt",
 "rlvnt_wrds" : 158
 },
 {
 "relevant_file" : "tech/066.txt",
 "rlvnt_wrds" : 234
 },
 {
 "relevant_file" : "tech/212.txt",
 "rlvnt_wrds" : 254
 },
 {
 "relevant_file" : "tech/135.txt",
 "rlvnt_wrds" : 410
 },
 {
 "relevant_file" : "tech/097.txt",
 "rlvnt_wrds" : 237
 },
 {
 "relevant_file" : "tech/180.txt",
 "rlvnt_wrds" : 266
 },
 {
 "relevant_file" : "tech/224.txt",
 "rlvnt_wrds" : 176
 },
 {
 "relevant_file" : "tech/145.txt",
 "rlvnt_wrds" : 227
 },
 {
 "relevant_file" : "tech/074.txt",
 "rlvnt_wrds" : 204
 },
 {
 "relevant_file" : "tech/350.txt",
 "rlvnt_wrds" : 256
 },
 {
 "relevant_file" : "tech/225.txt",
 "rlvnt_wrds" : 254
 },
 {
 "relevant_file" : "tech/004.txt",
 "rlvnt_wrds" : 173
 },
 {
 "relevant_file" : "tech/177.txt",
 "rlvnt_wrds" : 203
 },
 {
 "relevant_file" : "tech/259.txt",
 "rlvnt_wrds" : 166
 },
 {
 "relevant_file" : "tech/087.txt",
 "rlvnt_wrds" : 304
 },
 {
 "relevant_file" : "tech/353.txt",
 "rlvnt_wrds" : 236
 },
 {
 "relevant_file" : "tech/220.txt",
 "rlvnt_wrds" : 247
 },
 {
 "relevant_file" : "tech/312.txt",
 "rlvnt_wr

RelevantWordsNotNormalized = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(relevant_file),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(rlvnt_wrds),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(RelevantWordsNotNormalized),Some(Data(None,None,Some(List(Values(Map(relevant_file -> tech/381.txt, rlvnt_wrds -> 234)), Values(Map(relevant_file -> tech/003.txt, rlvnt_wrds -> 122)), Values(Map(relevant_file -> tech/114.txt, rlvnt_wrds -> 209)), Values(Map(relevant_file -> tech/186.txt, rlvnt_wrds -> 182)), Values(Map(relevant_file -> tech/182.txt, rlvnt_wrds -> 119)), Values(Map(r...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(relevant_file),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(rlvnt_wrds),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(RelevantWordsNotNormalized),Some(Data(None,None,Some(List(Values(Map(relevant_file -> tech/381.txt, rlvnt_wrds -> 234)), Values(Map(relevant_file -> tech/003.txt, rlvnt_wrds -> 122)), Values(Map(relevant_file -> tech/114.txt, rlvnt_wrds -> 209)), Values(Map(relevant_file -> tech/186.txt, rlvnt_wrds -> 182)), Values(Map(relevant_file -> tech/182.txt, rlvnt_wrds -> 119)), Values(Map(r...

**Les mots les plus courants dans l'ensemble des fichiers**

In [35]:
val most_repeated_words = word_in_doc.drop("count_files").orderBy(col("sum_words").desc)
most_repeated_words.show(10)

+----------+---------+
|group_word|sum_words|
+----------+---------+
|    people|      961|
|technology|      504|
|    mobile|      494|
|     users|      408|
|     games|      406|
|     music|      388|
|  software|      373|
|   digital|      373|
|       net|      371|
|      game|      326|
+----------+---------+
only showing top 10 rows



most_repeated_words = [group_word: string, sum_words: bigint]


[group_word: string, sum_words: bigint]

In [36]:
val word_in_doc_plot = Vegas("word_in_doc_plot")
    .withDataFrame(most_repeated_words.limit(100))
    .encodeX("group_word", Nom)
    .encodeY("sum_words", Quant)
    .mark(Bar)
render_vegas(word_in_doc_plot)

<iframe id="frame-vegas-03eea825-5496-4e5e-ae4d-aa94d2b83d83" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-03eea825-5496-4e5e-ae4d-aa94d2b83d83'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "field" : "group_word",
 "type" : "nominal"
 },
 "y" : {
 "field" : "sum_words",
 "type" : "quantitative"
 }
 },
 "description" : "word_in_doc_plot",
 "data" : {
 "values" : [
 {
 "group_word" : "people",
 "sum_words" : 961
 },
 {
 "group_word" : "technology",
 "sum_words" : 504
 },
 {
 "group_word" : "mobile",
 "sum_words" : 494
 },
 {
 "group_word" : "users",
 "sum_words" : 408
 },
 {
 "group_word" : "games",
 "sum_words" : 406
 },
 {
 "group_word" : "music",
 "sum_words" : 388
 },
 {
 "group_word" : "digital",
 "sum_words" : 373
 },
 {
 "group_word" : "software",
 "sum_words" : 373
 },
 {
 "group_word" : "net",
 "sum_words" : 371
 },
 {
 "group_word" : "game",
 "sum_words" : 326
 },
 {
 "group_word" : "computer",
 "sum_words" : 318
 },
 {
 "group_word" : "phone",
 "sum_words" : 318
 },
 {
 "group_word" : "online",
 "sum_words" : 309
 },
 {
 "group_word" : "time",
 "sum_words" : 307
 },
 {
 "group_word" : "uk",
 "sum_words" : 303
 },
 {
 "group_word" : "world",
 "sum_words" : 300
 },
 {
 "group_word" : "video",
 "sum_words" : 285
 },
 {
 "group_word" : "service",
 "sum_words" : 282
 },
 {
 "group_word" : "tv",
 "sum_words" : 282
 },
 {
 "group_word" : "microsoft",
 "sum_words" : 271
 },
 {
 "group_word" : "internet",
 "sum_words" : 254
 },
 {
 "group_word" : "million",
 "sum_words" : 253
 },
 {
 "group_word" : "broadband",
 "sum_words" : 245
 },
 {
 "group_word" : "using",
 "sum_words" : 242
 },
 {
 "group_word" : "data",
 "sum_words" : 242
 },
 {
 "group_word" : "information",
 "sum_words" : 239
 },
 {
 "group_word" : "services",
 "sum_words" : 233
 },
 {
 "group_word" : "system",
 "sum_words" : 231
 },
 {
 "group_word" : "phones",
 "sum_words" : 223
 },
 {
 "group_word" : "market",
 "sum_words" : 216
 },
 {
 "group_word" : "security",
 "sum_words" : 212
 },
 {
 "group_word" : "players",
 "sum_words" : 203
 },
 {
 "group_word" : "content",
 "sum_words" : 201
 },
 {
 "group_word" : "apple",
 "sum_words" : 198
 },
 {
 "group_word" : "firm",
 "sum_words" : 198
 },
 {
 "group_word" : "search",
 "sum_words" : 194
 },
 {
 "group_word" : "media",
 "sum_words" : 193
 },
 {
 "group_word" : "firms",
 "sum_words" : 193
 },
 {
 "group_word" : "web",
 "sum_words" : 191
 },
 {
 "group_word" : "according",
 "sum_words" : 191
 },
 {
 "group_word" : "based",
 "sum_words" : 190
 },
 {
 "group_word" : "pc",
 "sum_words" : 185
 },
 {
 "group_word" : "news",
 "sum_words" : 185
 },
 {
 "group_word" : "companies",
 "sum_words" : 182
 },
 {
 "group_word" : "research",
 "sum_words" : 182
 },
 {
 "group_word" : "mail",
 "sum_words" : 179
 },
 {
 "group_word" : "networks",
 "sum_words" : 176
 },
 {
 "group_word" : "site",
 "sum_words" : 174
 },
 {
 "group_word" : "company",
 "sum_words" : 172
 },
 {
 "group_word" : "set",
 "sum_words" : 172
 },
 {
 "group_word" : "bbc",
 "sum_words" : 169
 },
 {
 "group_word" : "sony",
 "sum_words" : 169
 },
 {
 "group_word" : "home",
 "sum_words" : 169
 },
 {
 "group_word" : "access",
 "sum_words" : 169
 },
 {
 "group_word" : "industry",
 "sum_words" : 167
 },
 {
 "group_word" : "help",
 "sum_words" : 165
 },
 {
 "group_word" : "own",
 "sum_words" : 164
 },
 {
 "group_word" : "virus",
 "sum_words" : 162
 },
 {
 "group_word" : "website",
 "sum_wo

word_in_doc_plot = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(group_word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(sum_words),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(word_in_doc_plot),Some(Data(None,None,Some(List(Values(Map(group_word -> people, sum_words -> 961)), Values(Map(group_word -> technology, sum_words -> 504)), Values(Map(group_word -> mobile, sum_words -> 494)), Values(Map(group_word -> users, sum_words -> 408)), Values(Map(group_word -> games, sum_words -> 406)), Values(Map(group_word -> music, sum_words -> 388)), Values(Map(group_word -> digital...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(None,None,None,Some(group_word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(sum_words),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(word_in_doc_plot),Some(Data(None,None,Some(List(Values(Map(group_word -> people, sum_words -> 961)), Values(Map(group_word -> technology, sum_words -> 504)), Values(Map(group_word -> mobile, sum_words -> 494)), Values(Map(group_word -> users, sum_words -> 408)), Values(Map(group_word -> games, sum_words -> 406)), Values(Map(group_word -> music, sum_words -> 388)), Values(Map(group_word -> digital...

**Obtenir le maximum tfidf et la somme du tfidf par mot**

In [37]:
var sum_max_tfidf = file_word_tfidf.groupBy("word")
    .agg(sum("tfidf").as("sum_tfidf"), max("tfidf").as("max_tfidf"))
sum_max_tfidf.show(10)

+----------+--------------------+--------------------+
|      word|           sum_tfidf|           max_tfidf|
+----------+--------------------+--------------------+
|    online|   1.468437597517171| 0.05573499800113514|
|       art|  0.3085495669921823|  0.0806679259199671|
|creativity| 0.04477508105227308|0.028808773080144696|
|   bricked|0.016287938661159157|0.016287938661159157|
|      hope| 0.16701683673477613| 0.02070238806795616|
| connected|  0.2982541247660457|0.030679740048255123|
|    tonnes|0.027369686882678396|0.027369686882678396|
|   barrier|  0.1598464373974418| 0.03155022507959422|
| amplifier| 0.04818922042496931|0.024094610212484654|
|       fog|0.031714081625960684|0.031714081625960684|
+----------+--------------------+--------------------+
only showing top 10 rows



sum_max_tfidf = [word: string, sum_tfidf: double ... 1 more field]


[word: string, sum_tfidf: double ... 1 more field]

In [38]:
sum_max_tfidf.count()

11263

**Juste un exercice pour visualiser le tfidf, et les résultats sont les attendus quant à la tendance, il parait qu'il n'y a aucune relation du mot avec l'ensemble puisque le tfidf met plutôt en relation le fait qu'un mot ne se repete pas dans d'autres documents. Par contre, nous pouvons voir qu'il parait qu'il y a des mots pour des documents qui sont plus caractéristiques que d'autres et on le voit pour la courve descendante.**

In [39]:
// https://github.com/vegas-viz/Vegas/blob/master/core/src/main/scala/vegas/package.scala
val scatter = Vegas("scatter")
    .withDataFrame(sum_max_tfidf.orderBy(col("sum_tfidf").desc).limit(100))
    .encodeX("word", Nom, sortField=Sort("sum_tfidf", AggOps.Max, SortOrder.Desc))
    .encodeY("sum_tfidf", Quant)
    .encodeSize("max_tfidf", Quant)
    .mark(Circle)
render_vegas(scatter)

<iframe id="frame-vegas-4b826643-d824-4eba-b9dd-0ee26385937c" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-4b826643-d824-4eba-b9dd-0ee26385937c'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "circle",
 "encoding" : {
 "x" : {
 "sort" : {
 "field" : "sum_tfidf",
 "op" : "max",
 "order" : "descending"
 },
 "field" : "word",
 "type" : "nominal"
 },
 "y" : {
 "field" : "sum_tfidf",
 "type" : "quantitative"
 },
 "size" : {
 "field" : "max_tfidf",
 "type" : "quantitative"
 }
 },
 "description" : "scatter",
 "data" : {
 "values" : [
 {
 "word" : "mobile",
 "sum_tfidf" : 2.8109838375143292,
 "max_tfidf" : 0.1119730290306753
 },
 {
 "word" : "games",
 "sum_tfidf" : 2.3377983542467216,
 "max_tfidf" : 0.12857897640232396
 },
 {
 "word" : "search",
 "sum_tfidf" : 2.2647504673461825,
 "max_tfidf" : 0.17735369933215675
 },
 {
 "word" : "game",
 "sum_tfidf" : 2.211340733152869,
 "max_tfidf" : 0.1053712568757636
 },
 {
 "word" : "microsoft",
 "sum_tfidf" : 2.110696425376912,
 "max_tfidf" : 0.11709163623262989
 },
 {
 "word" : "virus",
 "sum_tfidf" : 2.1062920081346577,
 "max_tfidf" : 0.20558978844659317
 },
 {
 "word" : "software",
 "sum_tfidf" : 2.08856093873931,
 "max_tfidf" : 0.057096558169347604
 },
 {
 "word" : "music",
 "sum_tfidf" : 2.080185221149715,
 "max_tfidf" : 0.10945697852048888
 },
 {
 "word" : "phone",
 "sum_tfidf" : 2.0585693509639498,
 "max_tfidf" : 0.11413659254680102
 },
 {
 "word" : "broadband",
 "sum_tfidf" : 2.0144282275668983,
 "max_tfidf" : 0.11928458719437354
 },
 {
 "word" : "security",
 "sum_tfidf" : 1.8295793614317146,
 "max_tfidf" : 0.08271851177273906
 },
 {
 "word" : "net",
 "sum_tfidf" : 1.82105999358088,
 "max_tfidf" : 0.09196769159040127
 },
 {
 "word" : "apple",
 "sum_tfidf" : 1.7743630209989476,
 "max_tfidf" : 0.15330829649654903
 },
 {
 "word" : "tv",
 "sum_tfidf" : 1.729560734011822,
 "max_tfidf" : 0.09810595451503105
 },
 {
 "word" : "sony",
 "sum_tfidf" : 1.7084280950399127,
 "max_tfidf" : 0.13568691893460488
 },
 {
 "word" : "spam",
 "sum_tfidf" : 1.6779708861299978,
 "max_tfidf" : 0.1777834013261166
 },
 {
 "word" : "users",
 "sum_tfidf" : 1.6766734947981772,
 "max_tfidf" : 0.06004784520062787
 },
 {
 "word" : "phones",
 "sum_tfidf" : 1.6478757447913297,
 "max_tfidf" : 0.09062727642959625
 },
 {
 "word" : "mail",
 "sum_tfidf" : 1.6440086799690046,
 "max_tfidf" : 0.14866566770226275
 },
 {
 "word" : "computer",
 "sum_tfidf" : 1.607616904482284,
 "max_tfidf" : 0.056123680250520794
 },
 {
 "word" : "digital",
 "sum_tfidf" : 1.5960843631469865,
 "max_tfidf" : 0.07292019190684176
 },
 {
 "word" : "service",
 "sum_tfidf" : 1.5251022872574014,
 "max_tfidf" : 0.07639509525185881
 },
 {
 "word" : "bt",
 "sum_tfidf" : 1.5198820609192396,
 "max_tfidf" : 0.2558648729254356
 },
 {
 "word" : "data",
 "sum_tfidf" : 1.4780677487909502,
 "max_tfidf" : 0.06671528433914087
 },
 {
 "word" : "online",
 "sum_tfidf" : 1.4684375975171715,
 "max_tfidf" : 0.05573499800113514
 },
 {
 "word" : "information",
 "sum_tfidf" : 1.4419651862691358,
 "max_tfidf" : 0.057085832266410445
 },
 {
 "word" : "uk",
 "sum_tfidf" : 1.4404767772643217,
 "max_tfidf" : 0.03790870577377434
 },
 {
 "word" : "million",
 "sum_tfidf" : 1.4211538851871142,
 "max_tfidf" : 0.052196267969317034
 },
 {
 "word" : "technology",
 "sum_tfidf" : 1.4150359377876822,
 "max_tfidf" : 0.034087941336954856
 },
 {
 "word" : "site",
 "sum_tfidf" : 1.4130754451072525,
 "max_tfidf" : 0.11855829400879238
 },
 {
 "word" : 

scatter = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortField(SortField(sum_tfidf,Max,Some(Descending)))),Some(word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(sum_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,Some(ChannelDefWithLegend(None,None,None,Some(max_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None)),None,Some(scatter),Some(Data(None,None,Some(List(Values(Map(word -> mobile, sum_tfidf -> 2.8109838375143292, max_tfidf -> 0.1119730290306753)), Values(Map(word -> games, sum_tfidf -> 2.3377983542467216, max_tfidf -> 0.12857897640232396)), Values(Map(word -> search,...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortField(SortField(sum_tfidf,Max,Some(Descending)))),Some(word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(sum_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,Some(ChannelDefWithLegend(None,None,None,Some(max_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None)),None,Some(scatter),Some(Data(None,None,Some(List(Values(Map(word -> mobile, sum_tfidf -> 2.8109838375143292, max_tfidf -> 0.1119730290306753)), Values(Map(word -> games, sum_tfidf -> 2.3377983542467216, max_tfidf -> 0.12857897640232396)), Values(Map(word -> search,...

In [40]:
var max_sum_tfidf = sum_max_tfidf.orderBy(col("max_tfidf").desc)
max_sum_tfidf.show(10)

+---------+-------------------+-------------------+
|     word|          sum_tfidf|          max_tfidf|
+---------+-------------------+-------------------+
|commodore| 0.9491909514014588| 0.4532131540511487|
|  posters|0.41208484812732665|0.41208484812732665|
|      fbi| 0.9040188893015384| 0.3880140687315098|
|    lifts| 0.3507105090445333| 0.3507105090445333|
|      uwb| 0.6483087835357592|0.33979578504786045|
|   virgin| 0.4936190607532068| 0.3348012335395775|
|      ink| 0.7275089894962428|0.32530335755505413|
|   raskin|0.30826087340433783|0.30826087340433783|
|      sun| 0.6895539866400031| 0.3075694195849576|
| argonaut| 0.5213453671707206| 0.3043529711050693|
+---------+-------------------+-------------------+
only showing top 10 rows



max_sum_tfidf = [word: string, sum_tfidf: double ... 1 more field]


[word: string, sum_tfidf: double ... 1 more field]

In [41]:
val scatter3 = Vegas("scatter3")
    .withDataFrame(sum_max_tfidf.orderBy(col("max_tfidf").desc).limit(100))
    .encodeX("word", Nom, sortField=Sort("max_tfidf", AggOps.Max, SortOrder.Desc))
    .encodeY("max_tfidf", Quant)
    .encodeSize("sum_tfidf", Quant)
    .mark(Circle)
render_vegas(scatter3)

<iframe id="frame-vegas-c594016d-e7f5-4876-af99-008e6e1ee402" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <div id='vegas-c594016d-e7f5-4876-af99-008e6e1ee402'></div>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "circle",
 "encoding" : {
 "x" : {
 "sort" : {
 "field" : "max_tfidf",
 "op" : "max",
 "order" : "descending"
 },
 "field" : "word",
 "type" : "nominal"
 },
 "y" : {
 "field" : "max_tfidf",
 "type" : "quantitative"
 },
 "size" : {
 "field" : "sum_tfidf",
 "type" : "quantitative"
 }
 },
 "description" : "scatter3",
 "data" : {
 "values" : [
 {
 "word" : "commodore",
 "sum_tfidf" : 0.9491909514014588,
 "max_tfidf" : 0.4532131540511487
 },
 {
 "word" : "posters",
 "sum_tfidf" : 0.41208484812732665,
 "max_tfidf" : 0.41208484812732665
 },
 {
 "word" : "fbi",
 "sum_tfidf" : 0.9040188893015384,
 "max_tfidf" : 0.3880140687315098
 },
 {
 "word" : "lifts",
 "sum_tfidf" : 0.3507105090445333,
 "max_tfidf" : 0.3507105090445333
 },
 {
 "word" : "uwb",
 "sum_tfidf" : 0.6483087835357592,
 "max_tfidf" : 0.33979578504786045
 },
 {
 "word" : "virgin",
 "sum_tfidf" : 0.4936190607532068,
 "max_tfidf" : 0.3348012335395775
 },
 {
 "word" : "ink",
 "sum_tfidf" : 0.7275089894962428,
 "max_tfidf" : 0.32530335755505413
 },
 {
 "word" : "raskin",
 "sum_tfidf" : 0.30826087340433783,
 "max_tfidf" : 0.30826087340433783
 },
 {
 "word" : "sun",
 "sum_tfidf" : 0.6895539866400031,
 "max_tfidf" : 0.3075694195849576
 },
 {
 "word" : "argonaut",
 "sum_tfidf" : 0.5213453671707206,
 "max_tfidf" : 0.3043529711050693
 },
 {
 "word" : "cabir",
 "sum_tfidf" : 0.5936034139336128,
 "max_tfidf" : 0.2896656295052343
 },
 {
 "word" : "robot",
 "sum_tfidf" : 0.6776283825790752,
 "max_tfidf" : 0.2871069660798985
 },
 {
 "word" : "library",
 "sum_tfidf" : 0.4752902065501929,
 "max_tfidf" : 0.2865137698689896
 },
 {
 "word" : "rfid",
 "sum_tfidf" : 0.5880687613069944,
 "max_tfidf" : 0.2849145001323971
 },
 {
 "word" : "musicians",
 "sum_tfidf" : 0.3286745992865313,
 "max_tfidf" : 0.28035039119955935
 },
 {
 "word" : "skype",
 "sum_tfidf" : 0.6189781020449222,
 "max_tfidf" : 0.2764600239712007
 },
 {
 "word" : "jacobsen",
 "sum_tfidf" : 0.2724527921502986,
 "max_tfidf" : 0.2724527921502986
 },
 {
 "word" : "linux",
 "sum_tfidf" : 0.7931655640989995,
 "max_tfidf" : 0.2684477254222323
 },
 {
 "word" : "tags",
 "sum_tfidf" : 0.5849856463572365,
 "max_tfidf" : 0.26817109909022996
 },
 {
 "word" : "patents",
 "sum_tfidf" : 0.7132660078621235,
 "max_tfidf" : 0.26791390852198177
 },
 {
 "word" : "yoran",
 "sum_tfidf" : 0.2678865442371651,
 "max_tfidf" : 0.2678865442371651
 },
 {
 "word" : "cafes",
 "sum_tfidf" : 0.34920302592889746,
 "max_tfidf" : 0.2635505937901484
 },
 {
 "word" : "domains",
 "sum_tfidf" : 0.5808936396972815,
 "max_tfidf" : 0.26307141089234815
 },
 {
 "word" : "espn",
 "sum_tfidf" : 0.259738328516618,
 "max_tfidf" : 0.259738328516618
 },
 {
 "word" : "bt",
 "sum_tfidf" : 1.5198820609192396,
 "max_tfidf" : 0.2558648729254356
 },
 {
 "word" : "skulls",
 "sum_tfidf" : 0.3223731010175872,
 "max_tfidf" : 0.25441106500417243
 },
 {
 "word" : "itunes",
 "sum_tfidf" : 0.5873224100189856,
 "max_tfidf" : 0.24190991449568777
 },
 {
 "word" : "bagle",
 "sum_tfidf" : 0.2564003710657988,
 "max_tfidf" : 0.24094610212484655
 },
 {
 "word" : "printer",
 "sum_tfidf" : 0.5654357128504448,
 "max_tfidf" : 0.24075487567074388
 },
 {
 "word" : "domain",
 "sum_tfidf" : 0.9896192131824719,
 "max_tfidf" : 0.2384794508993202
 },
 {
 "word" : "optinrealbig"

scatter3 = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortField(SortField(max_tfidf,Max,Some(Descending)))),Some(word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(max_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,Some(ChannelDefWithLegend(None,None,None,Some(sum_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None)),None,Some(scatter3),Some(Data(None,None,Some(List(Values(Map(word -> commodore, sum_tfidf -> 0.9491909514014588, max_tfidf -> 0.4532131540511487)), Values(Map(word -> posters, sum_tfidf -> 0.41208484812732665, max_tfidf -> 0.41208484812732665)), Values(Map(word ->...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(None,None,Some(SortSortField(SortField(max_tfidf,Max,Some(Descending)))),Some(word),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(max_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,Some(ChannelDefWithLegend(None,None,None,Some(sum_tfidf),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None)),None,Some(scatter3),Some(Data(None,None,Some(List(Values(Map(word -> commodore, sum_tfidf -> 0.9491909514014588, max_tfidf -> 0.4532131540511487)), Values(Map(word -> posters, sum_tfidf -> 0.41208484812732665, max_tfidf -> 0.41208484812732665)), Values(Map(word ->...